## Jupyter notebook to demonstrate the basic calibration setup

This notebook runs the following tasks:
* Create a test Visibility dataset. Single point source without corruptions or noise.

This notebook requires:
* See imports.

In [1]:
# Demonstration of basic calibration

# Imports
import warnings

import numpy as np
import xarray
from astropy import units
from astropy.coordinates import Angle, SkyCoord

# from ska_sdp_func_python.calibration.operations import apply_gaintable
from ska_low_mccs_calibration.utils import apply_gaintable
from ska_sdp_datamodels.calibration.calibration_create import (
    create_gaintable_from_visibility,
)
from ska_sdp_datamodels.configuration.config_create import (
    create_named_configuration,
)
from ska_sdp_datamodels.science_data_model import PolarisationFrame
from ska_sdp_datamodels.visibility.vis_create import create_visibility
from ska_sdp_func_python.calibration.solvers import solve_gaintable

warnings.simplefilter(action="ignore", category=FutureWarning)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
# Create a test Visibility dataset

# -------------------------------------------------------------------------- #
# Set up the array

# Read in an array configuration
low_config = create_named_configuration("LOWBD2")

# Down-select to a desired sub-array
#  - ECP-240228 modified AA2 clusters:
#      Southern Arm: S8 (x6), S9, S10 (x6), S13, S15, S16
#      Northern Arm: N8, N9, N10, N13, N15, N16
#      Eastern Arm: E8, E9, E10, E13.
#  - Most include only 4 of 6 stations, so just use the first 4:
AA2 = (
    np.concatenate(
        (
            345 + np.arange(6),  # S8-1:6
            351 + np.arange(4),  # S9-1:4
            429 + np.arange(6),  # S10-1:6
            447 + np.arange(4),  # S13-1:4
            459 + np.arange(4),  # S15-1:4
            465 + np.arange(4),  # S16-1:4
            375 + np.arange(4),  # N8-1:4
            381 + np.arange(4),  # N9-1:4
            471 + np.arange(4),  # N10-1:4
            489 + np.arange(4),  # N13-1:4
            501 + np.arange(4),  # N15-1:4
            507 + np.arange(4),  # N16-1:4
            315 + np.arange(4),  # E8-1:4
            321 + np.arange(4),  # E9-1:4
            387 + np.arange(4),  # E10-1:4
            405 + np.arange(4),  # E13-1:4
        )
    )
    - 1
)
mask = np.isin(low_config.id.data, AA2)
nstations = low_config.stations.shape[0]
low_config = low_config.sel(indexers={"id": np.arange(nstations)[mask]})

# Reset relevant station parameters
nstations = low_config.stations.shape[0]
low_config.stations.data = np.arange(nstations).astype("str")
low_config = low_config.assign_coords(id=np.arange(nstations))
# low_config.attrs["name"] = low_config.name+"-AA2"
low_config.attrs["name"] = "AA2-Low-ECP-240228"

print(f"Using {low_config.name} with {nstations} stations")

# -------------------------------------------------------------------------- #
# Set up the observation

# Set the phase centre in the ICRS coordinate frame
ra0 = Angle(0.0 * units.hourangle)
dec0 = Angle(-27.0 * units.deg)

# Set the parameters of sky model components
# chanwidth = 400e6 / 512  # station/CBF coarse channels = 781.25 kHz
chanwidth = 5.4e3  # Hz
nfrequency = 64
frequency = 100e6 + chanwidth * np.arange(nfrequency)
sample_time = 0.9  # seconds
solution_interval = sample_time  # would normally be minutes

# Set the phase centre hour angle range for the sim (in radians)
ha0 = 1 * np.pi / 12  # radians
ha = ha0 + np.arange(0, solution_interval, sample_time) / 3600 * np.pi / 12

# Create the Visibility dataset
vis = create_visibility(
    low_config,
    ha,
    frequency,
    channel_bandwidth=[chanwidth] * len(frequency),
    polarisation_frame=PolarisationFrame("linear"),
    phasecentre=SkyCoord(ra=ra0, dec=dec0),
    weight=1.0,
)

# Put a point source at phase centre
vis.vis.data[..., 0] = vis.vis.data[..., 3] = 1 + 0j

# Possible future development:
#  - Add thermal noise.
#  - Add more components.
#     - Use a skycomponents list.
#     - Use the GSM package.
#     - Use a DFT function.
#     - Image-based sky models with degridding?
#  - Include direction-independent gains and delays.
#  - Include beam models.
#  - Generate an ionospheric phase screen and add direction-dependent delays.
#  - Use the phase screen to add differential ionospheric Faraday rotation.

# Apply DI receiver gain and leakage terms?
#  - Ignore polarisation for now. Just get basic calibration working.
Rjones = create_gaintable_from_visibility(
    vis, jones_type="B", timeslice=solution_interval
)
g_sigma = 0.1
Rjones.gain.data[..., 0, 0] = np.random.normal(
    1, g_sigma, Rjones.gain.shape[:3]
)
Rjones.gain.data[..., 1, 1] = np.random.normal(
    1, g_sigma, Rjones.gain.shape[:3]
)
Rjones.gain.data[..., 0, 0] += 1j * np.random.normal(
    0, g_sigma, Rjones.gain.shape[:3]
)
Rjones.gain.data[..., 1, 1] += 1j * np.random.normal(
    0, g_sigma, Rjones.gain.shape[:3]
)
vis = apply_gaintable(vis=vis, gt=Rjones, inverse=False)

Using AA2-Low-ECP-240228 with 68 stations


In [3]:
# Helper functions (might move some to package)


def get_slice_lims(length, nslice):
    """Generate a list of slice index limits for n slices.

    Allow the final slice to be smaller if need be.
    """
    slice_lim0 = np.arange(0, length, int(np.ceil(length / nslice)))
    return np.stack((slice_lim0, np.append(slice_lim0[1:], length))).T

In [4]:
# Do pre-processing

# Get the LSM (single call for all channels / dask tasks)

# Adapative RFI flagging (assume known flags/birdies have been applied)
#  - Could also use dask parallelism.

# Chunking of Visibility dataset in frequency

# Averaging of Visibility datasets in time or frequency.
#  - Presumably use dask parallelism.
#  - Done as part of chunking?

In [15]:
# Predict model visibilities

# Could do this inside the bandpass calibration area, but do it here so that:
#  - it is available to other calibration workflows,
#  - different task distribution can be used, and
#  - accelerators can be used.

# Set a number of "parallel dask tasks" to divide predict between
#  - There is no dask. Each "task" is done sequentially.
ndasktask = 6

# Create a model Visibility dataset
#  - Is it better to generate separate sub-band datasets and then concatenate?
#     - It is presumably faster to allocate the xarray sub-bands in parallel.
#     - If they have frequency chunking anyway, perhaps concat is efficient...
#  - Want frequency chuncking, so copy rather than calling create_visibility?
#     - Seems reasonable, but may want to duplicate pre-proc time and
#       frequency averaging if there is appreciable decorrelation. In which
#       case this would need to be done before pre-precessing.
modelvis = vis.assign({"vis": xarray.zeros_like(vis.vis)})
assert np.all(modelvis.vis.data == 0)

print(
    "full spectral range: "
    + f"{np.min(modelvis.frequency.data)/1e6:.4f} - "
    + f"{np.max(modelvis.frequency.data)/1e6:.4f} MHz"
)

nchan = 0
for dasktask, slice_lims in enumerate(get_slice_lims(nfrequency, ndasktask)):
    bandmodel = modelvis.isel(frequency=slice(slice_lims[0], slice_lims[1]))
    nchan += len(bandmodel.frequency)
    print(
        "band spectral range: "
        + f"{np.min(bandmodel.frequency.data)/1e6:.4f} - "
        + f"{np.max(bandmodel.frequency.data)/1e6:.4f} MHz "
        + f"(dask task {dasktask}, nchan = {len(bandmodel.frequency)})"
    )

    # Put a point source at phase centre
    bandmodel.vis.data[..., 0] = bandmodel.vis.data[..., 3] = 1 + 0j

assert len(modelvis.frequency) == nchan

full spectral range: 100.0000 - 100.3402 MHz
band spectral range: 100.0000 - 100.0540 MHz (dask task 0, nchan = 11)
band spectral range: 100.0594 - 100.1134 MHz (dask task 1, nchan = 11)
band spectral range: 100.1188 - 100.1728 MHz (dask task 2, nchan = 11)
band spectral range: 100.1782 - 100.2322 MHz (dask task 3, nchan = 11)
band spectral range: 100.2376 - 100.2916 MHz (dask task 4, nchan = 11)
band spectral range: 100.2970 - 100.3402 MHz (dask task 5, nchan = 9)


In [12]:
# Do the bandpass calibration

# Set a number of "parallel dask tasks" to divide calibration between
#  - There is no dask. Each "task" is done sequentially.
ndasktask = 4

# Create a full-band bandpass calibration gain table
#  - Too small to bother with frequency chunking?
gaintable = create_gaintable_from_visibility(
    vis, jones_type="B", timeslice=solution_interval
)

# Make sure that the model was updated
#  - i.e. ensure that the isel in predict used reference semantics
assert np.all(modelvis.vis.data == 0) is False

refant = 0

for dasktask, slice_lims in enumerate(get_slice_lims(nfrequency, ndasktask)):
    bandvis = vis.isel(frequency=slice(slice_lims[0], slice_lims[1]))
    bandmodel = modelvis.isel(frequency=slice(slice_lims[0], slice_lims[1]))
    bandtable = gaintable.isel(frequency=slice(slice_lims[0], slice_lims[1]))
    print(
        "band spectral range: "
        + f"{np.min(bandmodel.frequency.data)/1e6:.4f} - "
        + f"{np.max(bandmodel.frequency.data)/1e6:.4f} MHz "
        + f"(dask task {dasktask}, nchan = {len(bandmodel.frequency)})"
    )

    # Call bandpass calibration function for this sub-band.
    #  - isel above assumes that vis and table have equal spectral axes. But
    #    this should not need to be the case. However, func-python solvers do
    #    solve either for each channel separately or for all channels
    #    together. So if there is an intermediate situation, a loop will be
    #    needed in the bandpass calibration function to deal with each output
    #    channel separately. Would also need to generalise the slicing above
    #    to use absolute quantities rather than channel indices.
    #  - Internally, the bandpass calibration function might call solvers
    #    multiple times.
    #  - The bandpass calibration function should return a list of bad
    #    antennas and bad channels.

    tmptable = solve_gaintable(
        vis=bandvis,
        modelvis=bandmodel,
        # gain_table=bandtable,
        # solver=solver,
        phase_only=False,
        niter=200,
        tol=1e-06,
        crosspol=False,
        normalise_gains=None,
        jones_type="B",
        timeslice=solution_interval,
        refant=refant,
    )

    # This doesn't update gaintable:
    # bandtable.gain.data = tmptable.gain.data
    # This does update gaintable:
    bandtable.gain.data[...] = tmptable.gain.data[...]

inputdata = gaintable.gain.data * np.exp(
    -1j * np.angle(gaintable.gain.data[:, [refant], :, :, :])
)

assert np.all(np.isclose(gaintable.gain.data, inputdata))

band spectral range: 100.0000 - 100.0810 MHz (dask task 0, nchan = 16)
band spectral range: 100.0864 - 100.1674 MHz (dask task 1, nchan = 16)
band spectral range: 100.1728 - 100.2538 MHz (dask task 2, nchan = 16)
band spectral range: 100.2592 - 100.3402 MHz (dask task 3, nchan = 16)


In [7]:
# Do post-processing

# Do any required interpolation.

# Estimate delays using the full GainTable (could distribute over antennas)

# Estimate differential Faraday rotation using the full GainTable (could
# distribute over antennas)

# Generate QA and flagging information
